In [1]:
import pandas as pd
import dill
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head(3)

,Engine_volume,Mileage,Doors,Prod_year,Cylinders,Airbags,Category,Leather_interior,Fuel_type,Gear_box_type,Drive_wheels,Wheel,Color
0,1.8,157859,4,2013,4.0,12,Sedan,Yes,Petrol,Automatic,Front,Left wheel,Black
1,1.3,187312,4,2010,4.0,0,Hatchback,Yes,Hybrid,Automatic,Front,Left wheel,Blue
2,1.8,108000,4,2012,4.0,8,Sedan,No,Petrol,Automatic,Rear,Left wheel,Silver


In [4]:
with open('DTreeReg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [5]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Engine_volume',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Engine_volume')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('Mileage',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Mileage')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('Doors',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Doors')),
                                                                 ('stand...
                                                                  FeatureSelector(column='Wheel')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Wheel')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('Color',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Color')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Color')),
                                                                 ('standard',
                                                                  StandardScaler())]))])),
                ('reg',
                 DecisionTreeRegressor(max_depth=40, max_features='auto',
                                       max_leaf_nodes=100, min_samples_leaf=10,
                                       min_samples_split=30,
                                       random_state=12))])

### Тренировочные предсказания для метрик:

In [6]:
X_train = pd.read_csv("X_train.csv")
Train_preds = pipeline.predict(X_train)

Train_pred_df = pd.DataFrame({'Train_preds': Train_preds})
Train_pred_df.to_csv("train_predictions.csv", index=None)

### Тестовые предсказания:

In [7]:
preds = pipeline.predict(X_test)

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [8]:
preds[:10]

array([ 3546.16590909,  4842.58317025, 16653.95144265, 16653.95144265,
       19671.27608696,  5366.44364937,  7309.19155465, 38484.02272727,
       22140.52380952,  3546.16590909])

In [9]:
y_train = pd.read_csv("y_train.csv")
preds = pd.read_csv("test_predictions.csv")
Train_preds = pd.read_csv("train_predictions.csv")

In [10]:
mse = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

r2_tr = round(r2_score(y_train, Train_preds), 3)
r2_tst = round(r2_score(y_test, preds), 3)
if r2_tr>r2_tst: ovf = round((((r2_tr-r2_tst)/r2_tr)*100),2)
if r2_tr<r2_tst: ovf = round((((r2_tst-r2_tr)/r2_tst)*100),2)

print(f"mean_squared_error:    {mse}")
print(f"mean_absolute_error:   {mae} ")
print(f"R2 Train:    {round(r2_score(y_train, Train_preds), 3)}")
print(f"R2 Test:    {round(r2_score(y_test, preds), 3)}")
print(f"Overfitting:    " + str(ovf) + "%")

mean_squared_error:    142293514.27456024
mean_absolute_error:   6864.921430855434 
R2 Train:    0.621
R2 Test:    0.563
Overfitting:    9.34%
